# Actividad práctica guiada: Desarrollo de un modelo de PLN en español con Hugging Face

**Módulo:** Modelos de Inteligencia Artificial / Procesamiento del Lenguaje Natural  
**Curso:** Curso de Especialización en Inteligencia Artificial y Big Data (FP)  

En esta actividad vas a **desarrollar y entrenar un modelo de análisis de sentimiento en español** utilizando:

- Python  
- La librería **Hugging Face Transformers**  
- La librería **datasets** para cargar datos  
- Un modelo preentrenado en español como base (transfer learning)

La actividad está pensada para seguirla paso a paso y, al mismo tiempo, que sirva como **material de estudio**.

## Objetivos de aprendizaje

Al finalizar esta práctica serás capaz de:

- Explicar qué es el **Procesamiento del Lenguaje Natural (PLN)** y qué tipos de tareas resuelve.
- Identificar las partes principales de un **pipeline de PLN** moderno:
  - Datos → Tokenizador → Modelo → Métricas → Predicciones.
- Entender qué es un **modelo preentrenado** y qué es el **fine-tuning**.
- Utilizar **Hugging Face** para:
  - Cargar un conjunto de datos en español.
  - Cargar un modelo preentrenado en español.
  - Tokenizar texto y preparar tensores para el modelo.
  - Entrenar (fine-tuning) un modelo de clasificación de texto.
  - Evaluar el modelo y usarlo para predecir sentimientos de frases nuevas.

## 0. Introducción teórica al PLN y a los Transformers

### ¿Qué es el PLN?

El **Procesamiento del Lenguaje Natural (PLN)** es el área de la Inteligencia Artificial que se encarga de que las máquinas puedan **entender, generar y manipular lenguaje humano** (texto o voz).

Algunas tareas típicas de PLN son:

- **Clasificación de texto:**  
  - Ejemplo: detectar si una opinión es *positiva*, *negativa* o *neutra*.
- **Análisis de sentimiento.**
- **Detección de entidades (NER):** nombres de personas, lugares, organizaciones…
- **Resumen automático de textos.**
- **Traducción automática.**
- **Pregunta-respuesta**, chatbots, etc.

### De los modelos clásicos a los Transformers

Históricamente se usaban modelos basados en:

- Bolsas de palabras (*bag-of-words*), n-gramas…
- Modelos estadísticos clásicos (Naive Bayes, SVM, etc.)

Actualmente, los modelos más utilizados son los **Transformers**, como:

- **BERT**, **RoBERTa**, **GPT**, etc.

Características clave de los Transformers:

- Trabajan con **representaciones vectoriales** de palabras (embeddings).
- Usan un mecanismo llamado **atención (attention)** para ponderar la importancia de cada palabra en el contexto de la frase.
- Se entrenan primero de forma **general** sobre grandes cantidades de texto (preentrenamiento) y luego se **ajustan (fine-tuning)** a tareas concretas como análisis de sentimiento, NER, etc.

### ¿Qué es Hugging Face?

**Hugging Face** es un ecosistema muy utilizado en PLN que ofrece:

- La librería **`transformers`**, con modelos preentrenados listos para usar.
- La librería **`datasets`**, con muchos conjuntos de datos listos para cargar.
- El **Hugging Face Hub**, un repositorio online donde se publican modelos y datasets.

En esta práctica usaremos:

- Un modelo en español: `pysentimiento/robertuito-sentiment-analysis`, preentrenado para análisis de sentimiento.
- Un dataset de opiniones en español: `pysentimiento/es_sentiment`.

## 1. Preparación del entorno

En esta sección vamos a:

1. Verificar la versión de Python.
2. Instalar las librerías necesarias.
3. Importar los módulos que utilizaremos.

> **Nota:** solo necesitas ejecutar las celdas de código (las que tienen `In [ ]:` a la izquierda).  
> Las celdas de tipo markdown (como esta) son texto explicativo.

In [1]:
# Comprobar la versión de Python (opcional)
import sys
print(sys.version)

3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]


In [8]:
# Si se trabaja en un entorno limpio, descomentar y ejecutar estas líneas para instalar las dependencias.
# En algunos entornos (como Google Colab) puede tardar unos minutos.

# !pip install transformers datasets accelerate evaluate -q

In [2]:
# Imports principales

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline,
)
import evaluate
import numpy as np

2025-11-25 11:01:26.793275: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 11:01:26.916650: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 11:01:29.724141: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 2. Cargar un dataset de sentimiento en español

Vamos a usar el dataset **`pysentimiento/es_sentiment`**, disponible en Hugging Face.  
Este conjunto de datos contiene textos en español etiquetados con sentimiento:

- `POS` → positivo  
- `NEG` → negativo  
- `NEU` → neutro  

### ¿Por qué necesitamos un dataset etiquetado?

Para entrenar un modelo de clasificación (supervisado) necesitamos:

- **Entradas**: textos en español (tweets, reseñas, opiniones…).
- **Salidas (etiquetas)**: la clase a la que pertenece cada texto (positiva, negativa, neutra).

El modelo aprenderá a aproximar una función:

\[
\text{texto} \longrightarrow \text{etiqueta (POS, NEG, NEU)}
\]

A la espera de concesión de permiso de acceso al REPO!! (25/11/2025)

In [3]:
# Cargar el dataset de sentimiento en español
dataset = load_dataset("pysentimiento/es_sentiment")

dataset

DatasetNotFoundError: Dataset 'pysentimiento/es_sentiment' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/pysentimiento/es_sentiment to ask for access.

In [ ]:
# Veamos un ejemplo del conjunto de entrenamiento
dataset["train"][0]

In [ ]:
# Inspeccionamos las columnas disponibles
dataset["train"].column_names

In [ ]:
# Tamaño de cada partición
print("Tamaño train:", len(dataset["train"]))
print("Tamaño validation:", len(dataset["validation"]))
print("Tamaño test:", len(dataset["test"]))

### Análisis rápido de la distribución de clases

Es buena práctica comprobar si el dataset está balanceado (si hay más ejemplos de una clase que de otra).

In [ ]:
from collections import Counter

label_names = dataset["train"].features["label"].names  # nombres de las etiquetas
print("Etiquetas:", label_names)

counter = Counter(dataset["train"]["label"])
for label_id, count in counter.items():
    print(f"Etiqueta {label_names[label_id]} ({label_id}): {count} ejemplos")

## 3. Tokenización: convertir texto en números

Los modelos tipo Transformer no trabajan directamente con texto, sino con **números**.

El proceso de conversión de texto a tokens numéricos se llama **tokenización**.  
Un **tokenizador** realiza varias tareas:

- Divide el texto en unidades (palabras o subpalabras).
- Asigna a cada token un **ID** entero.
- Añade tokens especiales de inicio/fin, si son necesarios.
- Gestiona la longitud máxima de la secuencia (`max_length`), truncando o rellenando (`padding`) cuando corresponde.

En Hugging Face utilizamos **`AutoTokenizer`** para cargar automáticamente el tokenizador adecuado para un modelo concreto.

In [ ]:
# Nombre del modelo preentrenado en español que vamos a usar
model_name = "pysentimiento/robertuito-sentiment-analysis"

# Cargar el tokenizador asociado al modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Probamos el tokenizador con una frase de ejemplo
ejemplo_texto = "Este curso de inteligencia artificial me parece increíble."
tokens = tokenizer(ejemplo_texto)

tokens

In [ ]:
# Para ver los tokens de forma más legible
print("Tokens:", tokenizer.convert_ids_to_tokens(tokens["input_ids"]))

### Preparar el dataset tokenizado

Ahora definimos una función que reciba un lote de ejemplos (batch) y devuelva su versión tokenizada.  
Usaremos `map` de `datasets` para aplicar esta función a todo el conjunto de datos.

Parámetros típicos del tokenizador:

- `truncation=True` → recorta textos demasiado largos.
- `padding="max_length"` o `padding=True` → rellena textos cortos hasta una longitud estándar.
- `max_length` → longitud máxima de tokens (por defecto, suele estar bien entre 128 y 256 para frases cortas).

In [ ]:
# Función de tokenización para aplicar al dataset completo
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding=False,  # el padding lo gestionaremos luego con el data collator
        max_length=128,
    )

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

tokenized_dataset

## 4. Definir el modelo de clasificación de texto

Vamos a cargar un modelo preentrenado en español para análisis de sentimiento y, además, vamos a ajustarlo (fine-tuning) usando nuestro dataset.

- Usamos **`AutoModelForSequenceClassification`** para cargar un modelo de clasificación de secuencias.
- Le indicamos cuántas etiquetas (`num_labels`) hay.
- Opcionalmente, podemos pasar el diccionario `id2label` y `label2id` para que las salidas sean más interpretables.

In [ ]:
# Definir el mapeo entre IDs y nombres de etiquetas
num_labels = len(label_names)
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

# Cargar el modelo de clasificación basado en el modelo preentrenado
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)

## 5. Preparar el entrenamiento (fine-tuning)

Para entrenar el modelo necesitamos:

1. **Datos tokenizados** en forma de tensores (input_ids, attention_mask, labels).
2. Un **data collator**, que se encargue de aplicar padding dinámico en cada batch.
3. Una configuración de entrenamiento (`TrainingArguments`):
   - Número de épocas (`num_train_epochs`).
   - Tamaño de batch (`per_device_train_batch_size`).
   - Tasa de aprendizaje (`learning_rate`).
   - Directorio de salida, etc.
4. Un **Trainer** que orqueste el proceso de entrenamiento y evaluación.

Además, definiremos métricas de evaluación, como:

- **Accuracy (exactitud)**.
- **F1** macro, útil cuando las clases están desbalanceadas.

In [ ]:
# Data collator: aplica padding dinámico para que los batch tengan la misma longitud
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Cargamos las métricas
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1_macro = f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
    }

In [ ]:
# Definimos los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="resultado_sentiment_es",
    evaluation_strategy="epoch",         # evaluar al final de cada época
    save_strategy="epoch",               # guardar modelo al final de cada época
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [ ]:
# Definimos el Trainer que gestionará el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

### Entrenar el modelo

> **Aviso:** esta celda puede tardar varios minutos en función del hardware disponible (CPU vs GPU).  
> Si estás en un entorno con GPU (por ejemplo, Google Colab), asegúrate de activarla.

In [ ]:
# Entrenamiento (fine-tuning)
train_result = trainer.train()

train_result

## 6. Evaluación del modelo

Una vez entrenado el modelo, es importante evaluarlo en el **conjunto de test**, que el modelo no ha visto durante el entrenamiento ni la validación.

Así obtenemos una estimación más realista de su rendimiento en datos nuevos.

In [ ]:
test_metrics = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
test_metrics

## 7. Uso del modelo entrenado para hacer predicciones

Vamos a probar el modelo con frases inventadas para ver qué sentimiento predice.  

Utilizaremos la función `pipeline` de Hugging Face, que simplifica mucho el proceso de inferencia.

In [ ]:
# Creamos un pipeline de análisis de sentimiento en español usando nuestro modelo entrenado
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=trainer.model,
    tokenizer=tokenizer,
)

# Probamos con algunas frases
frases = [
    "Este curso de inteligencia artificial me encanta, aprendo muchísimo.",
    "No me gusta nada este producto, es una pérdida de dinero.",
    "La película estuvo bien, pero tampoco fue espectacular.",
]

for frase in frases:
    resultado = sentiment_analyzer(frase)[0]
    print(f"Frase: {frase}")
    print(f"Predicción: {resultado['label']}, score={resultado['score']:.4f}")
    print("-" * 60)

## 8. Resumen teórico del flujo completo

A continuación se resume el flujo de trabajo que has seguido:

1. **Definición de la tarea de PLN**  
   - Clasificación de texto: análisis de sentimiento en español (POS, NEG, NEU).

2. **Selección de un modelo preentrenado**  
   - Modelo base: `pysentimiento/robertuito-sentiment-analysis`, entrenado previamente sobre grandes cantidades de texto en español (especialmente tweets).

3. **Carga del dataset etiquetado**  
   - Dataset: `pysentimiento/es_sentiment`, con ejemplos de texto en español y sus etiquetas de sentimiento.

4. **Preprocesamiento y tokenización**  
   - Conversión de texto a tokens e IDs numéricos con `AutoTokenizer`.
   - Aplicación de truncado y gestión de longitud máxima.

5. **Definición del modelo de clasificación**  
   - `AutoModelForSequenceClassification` añade una capa de clasificación encima del modelo base.
   - `num_labels`, `id2label` y `label2id` definen el espacio de salida.

6. **Configuración del entrenamiento (fine-tuning)**  
   - Parámetros de entrenamiento (épocas, batch size, learning rate…).
   - Definición de métricas (accuracy, F1 macro).
   - Uso de `Trainer` para gestionar el ciclo de entrenamiento y evaluación.

7. **Evaluación del modelo**  
   - Cálculo de métricas en el conjunto de test.

8. **Inferencia (uso en producción)**  
   - Creación de un `pipeline` de análisis de sentimiento para predecir etiquetas sobre nuevos textos.

Este esquema es muy similar para otras tareas de PLN con Transformers (NER, resumen, traducción, etc.), cambiando:

- El tipo de modelo y el `pipeline` (por ejemplo, `ner`, `summarization`, `translation`).
- El dataset y las etiquetas.

## 9. Actividades propuestas para afianzar el aprendizaje

1. **Cambiar el modelo base**  
   - Busca en Hugging Face otro modelo en español (por ejemplo, basado en RoBERTa o BERT multilingüe) y repite el proceso de entrenamiento.  
   - Compara las métricas obtenidas (accuracy, F1).

2. **Modificar hiperparámetros**  
   - Cambia el número de épocas (`num_train_epochs`), el tamaño de batch o la tasa de aprendizaje.  
   - Observa cómo afecta al rendimiento y al tiempo de entrenamiento.

3. **Análisis de errores**  
   - Filtra ejemplos donde el modelo falle (predicción distinta a la etiqueta real).  
   - Lee esos textos y analiza por qué crees que el modelo se equivoca.

4. **Ampliar el preprocesamiento**  
   - Añade una fase previa de limpieza del texto (por ejemplo, eliminar URLs, menciones, emojis…).  
   - Vuelve a entrenar y compara resultados.

5. **Crear tu propio pequeño dataset**  
   - Crea un CSV con frases en español y una etiqueta de sentimiento (por ejemplo, `POS` o `NEG`).  
   - Carga ese CSV con `load_dataset("csv", data_files=...)` y entrena un modelo con tus propios datos.

---

Con esta práctica tienes una **plantilla completa** para desarrollar modelos de PLN en español usando Hugging Face.  
A partir de aquí, puedes adaptar el código a otras tareas y datasets, manteniendo la misma estructura general.